In [15]:
import json

In [16]:
with open('mascqa-eval.json','r') as f:
    questions_file = json.load(f)
f.close()

In [17]:
qstr = questions_file['Material Applications']['qstr']
ques = questions_file['Material Applications']['questions']
nums = questions_file['Material Applications']['num_words']

# for qtype, q in zip(qstr, ques):
for i in range(len(nums)):
    if nums[i] < 35:
        print(i,ques[i],'\n')

3 Which one of the following metals is commonly alloyed with iron to improve its corrosion resistance?  (A) Co (B) Cr (C) Ti (D) Nb 

7 Nylon-6 is manufactured from  (A) caprolactum (B) adipic acid and hexamethylene diamine (C) maleic anhydride and hexamethylene diamine (D) sebasic acid and hexamethylene diamine 

8 The materials belonging to which one of the following crystal classes would be both piezoelectric and ferroelectric  (A) 222 (B) 4mm (C) -1 (D) 2/m 

9 Polymerized isotactic polybutadiene has a molecular weight of 3 x $10^5$ g/mol. The degree of polymerization is 

11 Given: ($M_(Ga)$ = 69.72 g/mol; $M_(As)$ = 74.92 g/mol). The weight in kg of gallium (Ga) to be mixed with arsenic (As) for obtaining 1.0 kg of gallium arsenide (GaAs) is 

13 To which of the following category of materials does Teflon (PTFE) belong?  (A) Thermosets (B) Thermoplastics (C) Elastomers (D) Block copolymers 

16 Defect-free single crystal alumina (sapphire) is  (A) opaque and white. (B) transparen

In [18]:
for q in questions_file['Material Applications']:
    ['qstr']

In [19]:
import pandas as pd
import os

df = pd.DataFrame(questions_file)
print(df)

                                              Thermodynamics  \
qids       [G-XEC-2012-3, G-XEC-2012-4, G-XEC-2012-6, G-X...   
qstr       [MCQS, MCQS, MCQS, MCQS, NUM, MCQS, MCQS, NUM,...   
num_words  [23, 28, 49, 19, 29, 25, 44, 50, 44, 50, 50, 7...   
questions  [Which of the following techniques is NOT used...   

                                            Atomic structure  \
qids       [G-XEC-2012-1, G-XEC-2012-2, G-XEC-2012-21, G-...   
qstr       [MCQS, MCQS, MCQS-NUM, MCQS-NUM, MCQS, MCQS, M...   
num_words  [24, 28, 33, 36, 50, 21, 18, 29, 35, 17, 31, 3...   
questions  [Which of the following is NOT a Bravais latti...   

                                                  Mechanical  \
qids       [G-XEC-2012-13, G-XEC-2012-19, G-XEC-2014-7, G...   
qstr       [MATCH, MCQS-NUM, MCQS, NUM, MCQS, NUM, MCQS, ...   
num_words  [79, 86, 45, 38, 53, 42, 30, 55, 22, 49, 19, 5...   
questions  [Match the properties in Column I with the opt...   

                                     

In [20]:
from openai import OpenAI

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

def ask_questions(questions):
    """
    Ask a list of questions to an LLM model like gpt-3.5-turbo and get the answers in a list format.

    Parameters:
    questions (list): A list of questions to ask the model.

    Returns:
    list: A list of answers provided by the model.
    """
    
    prompt = "Solve the following question. Write the correct answer inside a list at the end."
    
    answers = []
    for question in questions:
        response = client.chat.completions.create(
            model="gpt-4-0613",
            temperature=0,
            messages=[
                {"role": "assistant", "content":prompt},
                {"role": "user", "content": question}
            ]
        )
        
        # Extract the response from the model
        model_response = response.choices[0].message.content
        answers.append(model_response)

    return answers


# Example usage:
questions = [
    "What is the capital of France?",
    "What is 2 + 2?",
    "Who wrote 'To Kill a Mockingbird'?"
]

answers = ask_questions(questions)
answers

['The capital of France is Paris.\n\n- Paris',
 'The answer to 2 + 2 is 4.\n\nHere is the answer in a list:\n\n- 4',
 "'To Kill a Mockingbird' was written by Harper Lee.\n\n- Harper Lee"]

In [21]:
for i in questions_file.keys():
    newpath = f'./answer_gpt_4o-mini/{i}'
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    questions = df[f"{i}"]["questions"]
    answers = ask_questions(questions)
    for j in range(len(answers)):
        a = df[f"{i}"]["qids"][j]
        with open(f"./{newpath}/{a}.txt",'w') as file:
            file.write(answers[j])
    